In [112]:
import numpy as np
import pandas as pd

df_learning = pd.read_csv("../data-source/202208-202301.csv")
# df_learning = pd.read_csv("../data-source/201807-202207.csv")

In [113]:
import jieba

def cutReview(x): return ' '.join(jieba.lcut(str(x), cut_all=False))

df_learning.loc[:, '类型'] = df_learning['类型'].fillna(
    '')
# df_learning.loc[:, '交易对方'] = df_learning['交易对方'].fillna(
#     '').apply(cutReview)
# df_learning.loc[:, '商品名称'] = df_learning['商品名称'].fillna(
#     '').apply(cutReview)

df_learning.loc[:, '交易对方'] = df_learning['交易对方'].fillna(
    '')
df_learning.loc[:, '商品名称'] = df_learning['商品名称'].fillna(
    '')


# 模型训练

In [114]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from scipy.sparse import hstack

# 去除词频小于2的词
countvec = CountVectorizer(min_df=2)

# 构建特征向量
product_vector = countvec.fit_transform(df_learning['商品名称'])
saler_vector = countvec.fit_transform(df_learning['交易对方'])

# 合并特征向量
feature_vector = hstack((product_vector, saler_vector))

# 特征降维
pca = PCA(n_components=50)
feature_vector_pca = pca.fit_transform(feature_vector.toarray())

In [115]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 按照 7:3 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(
    feature_vector_pca, df_learning['类型'], test_size=0.3)

svm_model = SVC(kernel='rbf', probability=True, random_state=1)
svm_model.fit(x_train, y_train)

y_predict = svm_model.predict(x_test)

# 计算准确度
acc = accuracy_score(y_test, y_predict)
print("训练模型准确率为：{:.2f}%".format(acc * 100))


训练模型准确率为：61.25%


# 预测集填入

In [116]:
df_new = pd.read_csv("../out/merged.csv")

df_predict = df_new.copy()

def cutReview(x): return ' '.join(jieba.lcut(str(x), cut_all=False))

# df_predict.loc[:, '交易对方'] = df_predict['交易对方'].fillna(
#     '').apply(cutReview)
# df_predict.loc[:, '商品名称'] = df_predict['商品名称'].fillna(
#     '').apply(cutReview)

df_predict

,交易时间,类型,金额(元),收/支,支付方式,交易对方,商品名称,备注
0,2023-05-01 15:28:28,NaN,1000.00,收入,微信支付,冯裕浩,转账备注:老哥新婚快乐(⁠✷⁠‿⁠✷⁠),NaN
1,2023-05-01 15:28:45,NaN,1000.00,收入,微信支付,成家仪,转账备注:哈哈哈 新婚快乐！,NaN
2,2023-05-03 10:40:14,NaN,1000.00,收入,微信支付,华平叔叔,转账备注:成龙，祝愿你跟小张往后的日子美满、幸福。连理交枝，连枝相依,NaN
3,2023-05-03 18:24:34,NaN,10.00,支出,微信支付,芙蓉兴盛永盛便利店,芙蓉兴盛永盛便利店,NaN
4,2023-05-03 19:16:58,NaN,1.24,收入,微信支付,一叶子,/,NaN
...,...,...,...,...,...,...,...,...
119,2023-06-26 18:40:44,NaN,10.00,支出,微信支付,星之健身俱乐部有限公司,"健身卡-健身一天卡(￥10)(总额$10,需付$10)",NaN
120,2023-06-27 20:03:12,NaN,26.90,支出,支付宝,北京新发地菜篮子果蔬便民店,北京新发地菜篮子果蔬便民店,NaN
121,2023-06-28 19:55:11,NaN,47.56,支出,支付宝,沙河店,订单：677978867426,NaN
122,2023-06-28 20:00:28,NaN,4.00,支出,微信支付,小丽,收款方备注:二维码收款,NaN


In [117]:
# 去除词频小于2的词
countvec = CountVectorizer(min_df=2)

# 构建特征向量
product_vector = countvec.fit_transform(df_predict['商品名称'])
saler_vector = countvec.fit_transform(df_predict['交易对方'])

# 合并特征向量
feature_vector = hstack((product_vector, saler_vector))

# 特征降维
pca = PCA(n_components=50)
feature_vector_pca = pca.fit_transform(feature_vector.toarray())

# 结果预测

In [118]:
confidence_threshold = 0.6

y_predict = svm_model.predict(feature_vector_pca)
confidence_scores = svm_model.predict_proba(feature_vector_pca)

# 根据阈值过滤不可信的结果
for i in range(len(y_predict)):
    if np.max(confidence_scores[i]) < confidence_threshold:
        y_predict[i] = ''

df_new['类型'] = y_predict

df_new


,交易时间,类型,金额(元),收/支,支付方式,交易对方,商品名称,备注
0,2023-05-01 15:28:28,,1000.00,收入,微信支付,冯裕浩,转账备注:老哥新婚快乐(⁠✷⁠‿⁠✷⁠),NaN
1,2023-05-01 15:28:45,,1000.00,收入,微信支付,成家仪,转账备注:哈哈哈 新婚快乐！,NaN
2,2023-05-03 10:40:14,,1000.00,收入,微信支付,华平叔叔,转账备注:成龙，祝愿你跟小张往后的日子美满、幸福。连理交枝，连枝相依,NaN
3,2023-05-03 18:24:34,餐饮,10.00,支出,微信支付,芙蓉兴盛永盛便利店,芙蓉兴盛永盛便利店,NaN
4,2023-05-03 19:16:58,餐饮,1.24,收入,微信支付,一叶子,/,NaN
...,...,...,...,...,...,...,...,...
119,2023-06-26 18:40:44,交通,10.00,支出,微信支付,星之健身俱乐部有限公司,"健身卡-健身一天卡(￥10)(总额$10,需付$10)",NaN
120,2023-06-27 20:03:12,,26.90,支出,支付宝,北京新发地菜篮子果蔬便民店,北京新发地菜篮子果蔬便民店,NaN
121,2023-06-28 19:55:11,,47.56,支出,支付宝,沙河店,订单：677978867426,NaN
122,2023-06-28 20:00:28,,4.00,支出,微信支付,小丽,收款方备注:二维码收款,NaN


In [119]:
df_new.to_csv("../out/predict_result.csv", index=False, header=True, encoding='utf-8')